# Considerando teste para nenhum trade ou de 100 em 100 trades


In [8]:
from cvxopt.glpk import ilp
import cvxopt as cvx 
import numpy as np

def allocate_position_ratio(d_qt, p_qt, at_qt, ar_qt, var):          

        p_lim = 0.4883
        at_lim = 0.1307
        ar_lim = 0.0204

        lim = 0.001

        #define os limites percentuais das inequacoes
        p_lim_sup = p_lim + lim
        p_lim_inf = p_lim - lim
        at_lim_sup = at_lim + lim
        at_lim_inf = at_lim - lim
        ar_lim_sup = ar_lim + lim
        ar_lim_inf = ar_lim - lim

        #define os limites de quantidades
        p_less_qt = d_qt*p_lim_sup - p_qt
        p_more_qt = d_qt*p_lim_inf - p_qt
        at_less_qt = d_qt*at_lim_sup - at_qt
        at_more_qt = d_qt*at_lim_inf - at_qt
        ar_less_qt = d_qt*ar_lim_sup - ar_qt
        ar_more_qt = d_qt*ar_lim_inf - ar_qt

        #cost function
        #D + P + At + Ar + D_Trades + P_trades + At_Trades + Ar_Trades 

        c = cvx.matrix(np.array([1,1,1,1],dtype=float))
    
        #considera nao realizar nenhum trade e somente redistribuir entre os existentes
        if var == 0:
            h = cvx.matrix(np.array([p_less_qt,-p_more_qt
                                 ,at_less_qt,-at_more_qt
                                 ,ar_less_qt,-ar_more_qt],dtype=float))
            
            G = cvx.matrix(np.array([[-1*p_lim_sup,1,0,0],
                                 [1*p_lim_inf,-1,0,0],
                                 [-1*at_lim_sup,0,1,0],
                                 [1*at_lim_inf,0,-1,0],
                                 [-1*ar_lim_sup,0,0,1],
                                 [1*ar_lim_inf,0,0,-1]],dtype=float))
            
        #considera a adicao de 100 novas acoes, as redistribuindo entre os diferentes fundos
        else:
            #inequations
            h = cvx.matrix(np.array([p_less_qt,-p_more_qt
                                 ,at_less_qt,-at_more_qt
                                 ,ar_less_qt,-ar_more_qt
                                 ,0,0,0,0],dtype=float))

            G = cvx.matrix(np.array([[-1*p_lim_sup,1,0,0],
                                 [1*p_lim_inf,-1,0,0],
                                 [-1*at_lim_sup,0,1,0],
                                 [1*at_lim_inf,0,-1,0],
                                 [-1*ar_lim_sup,0,0,1],
                                 [1*ar_lim_inf,0,0,-1],
                                 [-1,0,0,0],
                                 [0,-1,0,0],
                                 [0,0,-1,0],
                                 [0,0,0,-1]],dtype=float))

        #equations
        b = cvx.matrix(np.array([var],dtype=float))
        A = cvx.matrix(np.array([[1,1,1,1]],dtype=float))

        (status,x) = ilp(c=c,G=G,A=A,b=b,h=h,I=set([0,1,2,3]))
        
        return x




In [10]:
#Quantidade de ações que cada fundo possui
d_qt = 172196
p_qt = 82536
at_qt = 21809
ar_qt = 3459

x = None
var = 0

while(x == None):
      
    x = allocate_position_ratio(d_qt, p_qt, at_qt, ar_qt, var)
    var= var + 100

print('A quantidade de ações que deverão ser alocadas no fundo Dartagnan são: ' + str(x[0])) 
print('A quantidade de ações que deverão ser alocadas no fundo Portos são: ' + str(x[1])) 
print('A quantidade de ações que deverão ser alocadas no fundo Athos são: ' + str(x[2])) 
print('A quantidade de ações que deverão ser alocadas no fundo Aramis são: ' + str(x[3]))
print('A quantidade de novos trades que deverão ser realizado são: ' + str(x[0] + x[1] + x[2] + x[3])) 

A quantidade de ações que deverão ser alocadas no fundo Dartagnan são: -1298.0
A quantidade de ações que deverão ser alocadas no fundo Portos são: 743.0
A quantidade de ações que deverão ser alocadas no fundo Athos são: 357.0
A quantidade de ações que deverão ser alocadas no fundo Aramis são: 198.0
A quantidade de novos trades que deverão ser realizado são: 0.0


# Considerar novos trades de 100 ações

In [7]:
var = 100
x = None
while(x == None):
      
    x = allocate_position_ratio(d_qt, p_qt, at_qt, ar_qt, var)
    var= var + 100
    
print('A quantidade de ações que deverão ser alocadas no fundo Dartagnan são: ' + str(x[0])) 
print('A quantidade de ações que deverão ser alocadas no fundo Portos são: ' + str(x[1])) 
print('A quantidade de ações que deverão ser alocadas no fundo Athos são: ' + str(x[2])) 
print('A quantidade de ações que deverão ser alocadas no fundo Aramis são: ' + str(x[3]))
print('A quantidade de novos trades que deverão ser realizado são: ' + str(x[0] + x[1] + x[2] + x[3])) 

A quantidade de ações que deverão ser alocadas no fundo Dartagnan são: 0.0
A quantidade de ações que deverão ser alocadas no fundo Portos são: 1376.0
A quantidade de ações que deverão ser alocadas no fundo Athos são: 525.0
A quantidade de ações que deverão ser alocadas no fundo Aramis são: 99.0
A quantidade de novos trades que deverão ser realizado são: 2000.0
